In [ ]:
import geopandas
import leafmap
import geoplot
import geoplot.crs as gcrs
import pandas as pd
import matplotlib.pyplot as plt

This file is used to convert data from excel format to geopandas dataframe. The notebook contains also some plots and analysis of the data.

In [ ]:
aoi_dir = "data/aoi.geojson"
points = "data/measurements/shp_GA_5513_1739_2022/GA_5513_1739_2022_punkty_czynne_i_archiwalne_m_ilosc_i_bad.shp"

In [ ]:
geodf = geopandas.read_file(aoi_dir)
points_df = geopandas.read_file(points)
points_df = points_df.set_crs("EPSG:2180")
points_df = points_df.to_crs("EPSG:4326")
points_df['geometry'] = points_df['geometry'].explode(index_parts=False)

In [ ]:
points_df.head(2)

In [ ]:
ax = geoplot.polyplot(geodf, projection=gcrs.AlbersEqualArea(), edgecolor='darkgrey', facecolor='lightgrey', linewidth=.3,
    figsize=(12, 8))
geoplot.pointplot(points_df, ax=ax)

In [ ]:
def excel_to_geopandas(excel_data_dir, points_data_dir):
    excel_data = pd.read_excel(excel_data_dir)
    excel_data = excel_data.drop(excel_data.index[0])
    excel_data.columns = excel_data.iloc[0]
    excel_data = excel_data.drop(excel_data.index[0])
    excel_data.reset_index(inplace=True)
    excel_data.drop("index", inplace=True, axis=1)
    excel_data = excel_data.rename(columns = {"Data/Punkty": "date"})

    excel_data["date"] = pd.to_datetime(excel_data["date"], format='%Y-%m-%d %H:%M:%S')
    excel_data[excel_data.loc[:, excel_data.columns != "date"].columns] = excel_data[excel_data.loc[:, excel_data.columns != "date"].columns].astype('float64')
    excel_data_melted = excel_data.melt(id_vars=["date"],
                             var_name="name",
                             value_name="value")
    points_data = pd.read_excel(points_data_dir)
    points_data = points_data[["Numer punktu monitoringu stanu ilościowego", "Rzędna terenu [m n.p.m.]", "Współrzędne PUWG 1992 X", "Współrzędne PUWG 1992 Y"]]
    points_data = points_data.rename(columns = {"Numer punktu monitoringu stanu ilościowego": "name", "Rzędna terenu [m n.p.m.]": "height", "Współrzędne PUWG 1992 X": "longitude", "Współrzędne PUWG 1992 Y": "latitude"}) 
    data_merged = pd.merge(excel_data_melted, points_data, on="name")
    data_merged["normalized_value"] = data_merged["height"] - data_merged["value"] 
    gdf = geopandas.GeoDataFrame(data_merged, geometry=geopandas.points_from_xy(data_merged.longitude, data_merged.latitude))
    
    gdf = gdf.set_crs("EPSG:2180")
    gdf = gdf.to_crs("EPSG:4326")
    gdf = gdf.drop(["longitude", "latitude"], axis=1)
    return gdf
    

In [ ]:
excel_data_dirs = ["data/measurements/GA_5513_1739_2022_monitoring_st_ilosciowego_pomiary_manualne.xls", "data/measurements/GA_5513_1739_2022_monitoring_st_ilosciowego_pomiary_automatyczne.xls"]
excel_data_geojson_dirs = ["data/measurements/GA_5513_1739_2022_quantitative_status_monitoring_manual.json", "data/measurements/GA_5513_1739_2022_quantitative_status_monitoring_automatic.json"]
excel_data = pd.read_excel(excel_data_dirs[0])
excel_data.head()

In [ ]:
points_data_dir = "data/measurements/GA_5513_1739_2022_dane_o_punktach_czynnych_i_archiwalnych.xls"

In [ ]:
#Uncomment to generate geojson files
#for i, excel_data in enumerate(excel_data_dirs):
#    gdf = excel_to_geopandas(excel_data, points_data_dir)
#    gdf.to_file(excel_data_geojson_dirs[i], driver="GeoJSON")

In [ ]:
gdf_manual = excel_to_geopandas(excel_data_dirs[0], points_data_dir)
gdf_auto = excel_to_geopandas(excel_data_dirs[1], points_data_dir)

gdf_manual = gdf_manual.dropna()
gdf_auto = gdf_auto.dropna()

        
    

Some plots & correlation matrix

In [ ]:
#auto stations are continuation of manual stations
plt.rcParams["figure.figsize"] = (20,10)
fig, axs = plt.subplots(1, 2)
ax = axs[0]
for name in gdf_auto['name'].unique():
    gdf[gdf['name'] == name].plot(kind='line',x='date',y='value',ax=ax, c="black", label="Concatenated", style='-', linewidth=8)
    gdf_manual[gdf_manual['name'] == name].plot(kind='line',x='date',y='value', c="blue", ax=ax, label="Manual", style='-', linewidth=4)
    gdf_auto[gdf_auto['name'] == name].plot(kind='line',x='date',y='value',ax=ax, c="red", label="Automatic", style='-')

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

ax = axs[1]
for name in gdf_auto['name'].unique():
    gdf[gdf['name'] == name].plot(kind='line',x='date',y='normalized_value',ax=ax, c="black", label="Concatenated", style='-', linewidth=8)
    gdf_manual[gdf_manual['name'] == name].plot(kind='line',x='date',y='normalized_value', c="blue", ax=ax, label="Manual", style='-', linewidth=4)
    gdf_auto[gdf_auto['name'] == name].plot(kind='line',x='date',y='normalized_value',ax=ax, c="red", label="Automatic", style='-')

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())

plt.show()

In [ ]:
fig, ax = plt.subplots()
for name in gdf_auto['name'].unique():
    gdf[gdf['name'] == name].plot(kind='line',x='date',y='normalized_value',ax=ax, c="black", label="Concatenated", style='-', linewidth=8)
    gdf_manual[gdf_manual['name'] == name].plot(kind='line',x='date',y='normalized_value', c="blue", ax=ax, label="Manual", style='-', linewidth=4)
    gdf_auto[gdf_auto['name'] == name].plot(kind='line',x='date',y='normalized_value',ax=ax, c="red", label="Automatic", style='-')

handles, labels = ax.get_legend_handles_labels()
by_label = dict(zip(labels, handles))
ax.legend(by_label.values(), by_label.keys())
plt.show()

In [ ]:
corr = gdf.set_index('date')
corr = corr.pivot(columns='name', values='normalized_value')
corr = corr.corr()
#plot correlation matrix
plt.matshow(corr)
plt.xticks(range(len(corr.columns)), corr.columns, rotation='vertical')
plt.yticks(range(len(corr.columns)), corr.columns)
plt.colorbar()
plt.show()

In [ ]:
m = leafmap.Map()
m.add_gdf(gdf.drop(["date", "value"], axis=1).drop_duplicates(subset="geometry", keep="first"), layer_name="Stations", info_mode="on_click")
m.add_geojson("data/aoi.geojson", layer_name="AOI", info_mode="on_click")
m